In [ ]:
# Model experiments: Two-Tower Neural Network
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from src.models.ranking_model import build_two_tower_model
from src.features.feature_engineering import generate_features
from src.utils.evaluation import ndcg_at_k, precision_at_k
import numpy as np

# --- 1. Load features ---
df, user_features, item_features = generate_features()

# Encode user/item IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['user_enc'] = user_encoder.fit_transform(df['userId'])
df['item_enc'] = item_encoder.fit_transform(df['movieId'])

num_users = df['user_enc'].nunique()
num_items = df['item_enc'].nunique()

# --- 2. Build model ---
embedding_dim_list = [32, 64]  # Experimenting with embedding sizes
results = []

for dim in embedding_dim_list:
    model = build_two_tower_model(num_users, num_items)
    print(f"\nTraining Two-Tower Model with embedding_dim={dim} ...")
    
    # Train
    model.fit(
        [df['user_enc'], df['item_enc']],
        df['rating_norm'],
        batch_size=64,
        epochs=3,
        validation_split=0.1,
        verbose=1
    )
    
    # Predict for evaluation
    preds = model.predict([df['user_enc'], df['item_enc']], verbose=0)
    
    # Evaluate using NDCG@10 and Precision@10
    ndcg = ndcg_at_k(df['rating_norm'].values, preds.flatten(), k=10)
    prec = precision_at_k(df['rating_norm'].values, preds.flatten(), k=10)
    print(f"NDCG@10: {ndcg:.4f}, Precision@10: {prec:.4f}")
    
    results.append({'embedding_dim': dim, 'ndcg@10': ndcg, 'precision@10': prec})

# --- 3. Summary ---
results_df = pd.DataFrame(results)
print("\nSummary of experiments:")
print(results_df)

# Optional: Plot results
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
plt.plot(results_df['embedding_dim'], results_df['ndcg@10'], marker='o', label='NDCG@10')
plt.plot(results_df['embedding_dim'], results_df['precision@10'], marker='o', label='Precision@10')
plt.xlabel("Embedding Dimension")
plt.ylabel("Metric")
plt.title("Model Experiment Results")
plt.legend()
plt.show()
